# 多个function call并列情况下是否能正常工作

看起来是不能正常工作。

出现了执行一个函数后又去执行了另一个函数。

In [1]:
%%time

# 加载llm和embeddings
%run ../utils.py

CPU times: user 3.36 s, sys: 340 ms, total: 3.7 s
Wall time: 3.33 s


In [ ]:
%%time

from llama_index.core.tools import BaseTool, FunctionTool

In [16]:
%%time

from llama_index.core.tools import FunctionTool

def get_news(url: str) -> str:
    """根据url获取新闻内容"""
    return f"新闻内容来源于{url}"

get_news_tool = FunctionTool.from_defaults(
            fn=get_news)

def query(query: str) ->str:
    """根据查询提示词得到信息"""

    content="安全事故频发、“吹哨人”被曝遭到报复，让美国波音公司深陷舆论和信任危机。"
    return f"根据{query}查询到信息\n\n{content}"

query_tool = FunctionTool.from_defaults(
            fn=query)

CPU times: user 594 µs, sys: 0 ns, total: 594 µs
Wall time: 597 µs


In [17]:
%%time

from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
            [get_news_tool,query_tool], verbose=True)

CPU times: user 141 µs, sys: 16 µs, total: 157 µs
Wall time: 159 µs


In [18]:
%%time

response = agent.stream_chat("https://www.guancha.cn/internation/2024_06_19_738488.shtml")
response.print_response_stream()

Thought: The user has provided a URL to an article. I need to use the 'get_news' tool to extract the content of this article.
Action: get_news
Action Input: {'url': 'https://www.guancha.cn/internation/2024_06_19_738488.shtml'}
Observation: 新闻内容来源于https://www.guancha.cn/internation/2024_06_19_738488.shtml
Thought: I have successfully retrieved the content of the article using the 'get_news' tool. Now, I need to read and understand the content before providing an answer in the user's language.
Action: query
Action Input: {'query': '新闻内容'}
Observation: 根据新闻内容查询到信息

安全事故频发、“吹哨人”被曝遭到报复，让美国波音公司深陷舆论和信任危机。
 这篇文章讨论了波音公司因为安全事故频发以及“吹哨人”遭受报复事件，导致该公司陷入了公众关注和信任危机。CPU times: user 625 ms, sys: 123 ms, total: 748 ms
Wall time: 5.66 s
